In [ ]:
from collections import defaultdict
import numpy as np
from chempy import Equilibrium, Species
from chempy.equilibria import EqSystem, NumSysLin
from pyneqsys.symbolic import SymbolicSys
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
eqw = Equilibrium({'H2O': 1}, {'OH-': 1, 'H+': 1}, 1e-14)
eq_FeOH = Equilibrium({'Fe+3': 1, 'H2O': 1}, {'FeOH+2': 1, 'H+': 1}, 10**-2.4) #10**-2.774)
eq_Fe2OH2 = Equilibrium({'Fe+3': 2, 'H2O': 2}, {'Fe2(OH)2+4': 1, 'H+': 2}, 10**-2.9) #10**-2.81)
eq_FeSCN1 = Equilibrium({'Fe+3': 1, 'SCN-': 1}, {'FeSCN+2': 1}, 10**2.065)
eq_FeSCN2 = Equilibrium({'Fe+3': 1, 'SCN-': 2}, {'Fe(SCN)2+': 1}, 10**3.34)
eq_FeSCN3 = Equilibrium({'Fe+3': 1, 'SCN-': 3}, {'Fe(SCN)3': 1}, 10**3.82)

eqsys = EqSystem([eqw, eq_FeOH, eq_Fe2OH2, eq_FeSCN1, eq_FeSCN2, eq_FeSCN3],
                 'Fe+3 H2O FeOH+2 H+ Fe2(OH)2+4 OH- SCN- FeSCN+2 Fe(SCN)2+ Fe(SCN)3',
                 substance_factory=Species.from_formula)
c0 = defaultdict(float, {'H+': 0.05, 'Fe+3': 1e-3, 'H2O': 1, 'SCN-': .5e-3})

In [ ]:
numsys = NumSysLin(eqsys)
neqsys = SymbolicSys.from_callback(numsys.f, eqsys.ns, nparams=eqsys.ns + eqsys.nr)
neqsys.exprs

In [ ]:
x, info, sane = eqsys.root(c0)
assert sane
eqsys.as_per_substance_dict(x)

In [ ]:
plt.figure(figsize=(16, 8))
Cout, info, sanity = eqsys.roots(c0, np.logspace(-4, -1), 'SCN-', plot_kwargs={
        'substances': 'Fe+3 FeOH+2 H+ Fe2(OH)2+4 FeSCN+2 Fe(SCN)2+ Fe(SCN)3 SCN-'.split()})
#plt.gca().set_xscale('linear')

In [ ]:
SCNtot = (Cout[:, eqsys.as_substance_index('SCN-')] + Cout[:, eqsys.as_substance_index('FeSCN+2')] +
          2*Cout[:, eqsys.as_substance_index('Fe(SCN)2+')] + 3*Cout[:, eqsys.as_substance_index('Fe(SCN)3')])

In [ ]:
sol_prod = Cout[:, eqsys.as_substance_index('Fe+3')]*Cout[:, eqsys.as_substance_index('H+')]**-3
plt.loglog(SCNtot, sol_prod)
gamma_prod = 10
plt.loglog(SCNtot[[0, -1]], [gamma_prod*0.4]*2)

In [ ]:
FeSCN_2_and_3 = Cout[:, eqsys.as_substance_index('Fe(SCN)2+')] + Cout[:, eqsys.as_substance_index('Fe(SCN)3')]
plt.figure(figsize=(16, 6))
plt.subplot(1, 2, 1)
plt.plot(SCNtot, Cout[:, 7]*5148)
plt.xlabel('[SCN^-]_tot')
plt.ylabel('Absorbance')
plt.subplot(1, 2, 2)
plt.plot(SCNtot, FeSCN_2_and_3/(Cout[:, 7]+FeSCN_2_and_3))